In [ ]:
import sys
sys.path.insert(0, "../pycre/")

import time

from cre import *
from parsers import get_parser
from dataset import dataset_generator

In [7]:
args = get_parser().parse_args(args=[])

data_sizes = np.logspace(2, 3, 10).astype(int)
times = []
for n in data_sizes:
    print(n)
    start_time = time.time()
    X, y, z, ite = dataset_generator(effect_size=5,
                                    n_rules = 2,
                                    binary_out=False,
                                    n = n)
    args.verbose = False

    model = CRE(args)
    model.fit(X, y, z)
    end_time = time.time()

    execution_time = end_time - start_time
    print(f"Execution time: {execution_time} seconds")
    times.append(execution_time)

usage: ipykernel_launcher.py [-h] [--seed SEED] [--ratio_dis RATIO_DIS]
                             [--n_trees N_TREES] [--max_depth MAX_DEPTH]
                             [--node_size NODE_SIZE] [--max_rules MAX_RULES]
                             [--criterion CRITERION] [--decimal DECIMAL]
                             [--t_ext T_EXT] [--t_corr T_CORR] [--t_ss T_SS]
                             [--alphas ALPHAS] [--B B] [--subsample SUBSAMPLE]
                             [--learner_y LEARNER_Y] [--learner_ps LEARNER_PS]
                             [--method METHOD] [--exp_name EXP_NAME]
                             [--verbose VERBOSE] [--path PATH] [--save SAVE]
                             [--xrange XRANGE]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"e16d0d5d-2161-4a24-9722-64fb6c7fed50" --shell=9002 --transport="tcp" --iopub=9004 --f=/Users/riccardocadei/Librar

SystemExit: 2

In [ ]:
python_times = times
# compute in R 
R_times = [11.70290,  10.01399,  10.61894,  12.25200,  16.35832,  28.50032,  51.95816, 119.32511, 314.55477,  844.31094]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(data_sizes, python_times)
plt.plot(data_sizes, R_times)
plt.xlabel("Number of Individuals (n)")
plt.ylabel("Execution time (seconds)")
plt.xscale("log")
#plt.yscale("log")
yticks = np.arange(0, 1000, 100)
plt.yticks(yticks, yticks)
plt.legend(["Python", "R"]);